# 일괄 처리 엔드포인트에 배포

병원에서 하루 종일 환자의 상태를 측정하여 각 환자의 세부 정보를 개별 파일에 저장한다고 가정해 보겠습니다. 그리고 야간에는 당뇨병 예측 모델을 사용해 하루 동안 수집한 환자 데이터를 일괄 처리하여 예측을 생성할 수 있습니다. 그러면 당뇨 의심 대상으로 예측된 환자에 대해 다음날 후속 조치를 취할 수 있습니다. Azure Machine Learning을 사용하면 일괄 처리 엔드포인트를 만들어 이 작업을 수행할 수 있습니다. 이 연습에서 구현할 내용입니다.

## 시작하기 전에

이 Notebook에서 코드를 실행하려면  **최신 버전의 azureml-ai-ml** 패키지가 필요합니다. 아래 셀의 명령을 실행하여 이 패키지가 설치되어 있는지 확인합니다.

> **고**:
> **azure-ai-ml** 패키지가 설치되지 않은 경우 를 실행 `pip install azure-ai-ml` 하여 설치합니다.

In [ ]:
## 작업 영역에 연결

필요한 SDK 패키지를 설치했으므로 작업 영역에 연결할 수 있습니다.

작업 영역에 연결하려면 구독 ID, 리소스 그룹 이름 및 작업 영역 이름 등 식별자 매개 변수가 필요합니다. 리소스 그룹 이름 및 작업 영역 이름이 이미 채워져 있습니다. 명령을 완료하려면 구독 ID만 필요합니다.

필요한 매개 변수를 찾으려면 Studio의 오른쪽 위에 있는 구독 및 작업 영역 이름을 클릭합니다. 오른쪽에 창이 열립니다.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 구독 ID를 복사하고 **YOUR-SUBSCRIPTION-ID** 를 복사한 값으로 바꿉니다. </p>

## 모델 등록

일괄 배포는 작업 영역에 등록된 모델만 배포할 수 있습니다. 로컬 `model` 폴더에 저장된 MLflow 모델을 등록합니다. 

In [ ]:
## 일괄 처리 엔드포인트 만들기

일괄 처리 엔드포인트는 애플리케이션이 일괄 처리 채점 작업을 트리거하기 위해 호출할 수 있는 HTTPS 엔드포인트입니다. 일괄 처리 엔드포인트 이름은 Azure 지역 내에서 고유해야 합니다. 함수를 `datetime` 사용하여 현재 날짜 및 시간에 따라 고유한 이름을 생성합니다. 

In [ ]:
클래스를 사용하여 엔드포인트를 `BatchEndpoint` 만들려면 이름과 설명(선택 사항)을 지정해야 합니다. 엔드포인트를 만든 후에는 모델을 엔드포인트에 배포합니다.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 중요! 계속하기 전에 엔드포인트가 생성될 때까지 기다립니다. 스튜디오에 녹색 알림이 표시됩니다. </p>

## 배포 만들기

배포는 실제 유추를 수행하는 모델을 호스팅하는 데 필요한 리소스의 세트입니다. `BatchDeployment` 클래스를 사용하여 엔드포인트에 대한 배포를 만듭니다. 

MLflow 모델을 배포하기 때문에 점수 매기기 스크립트가 필요하거나 환경을 정의할 필요가 없습니다. Azure Machine Learning은 자동으로 해당 자산을 만듭니다. 폴더의 `model` 파일은 `MLmodel` 모델의 예상 입력 및 출력을 이해하는 데 사용됩니다.

다음 매개 변수를 사용하여 모델을 배포합니다.

- `name`: 배포의 이름입니다.
- `description`: 배포가 나타내는 내용을 더 명확히 하기 위한 선택적 설명입니다.
- `endpoint_name`: 모델을 배포해야 하는 이전에 만든 엔드포인트의 이름입니다.
- `model`: 등록된 모델의 이름입니다.
- `compute`: 예측을 생성하기 위해 배포된 모델을 호출할 때 사용할 컴퓨팅입니다.
- `instance_count`: 예측을 생성하는 데 사용할 컴퓨팅 노드 수입니다.
- `max_concurrency_per_instance`: 컴퓨팅 노드당 최대 병렬 채점 스크립트 실행 수입니다.
- `mini_batch_size`: 채점 스크립트 실행당 전달된 파일 수입니다.
- `output_action`: 각 새 예측은 출력 파일에 새 행으로 추가됩니다.
- `output_file_name`: 예측을 추가할 파일입니다.
- `retry_settings`: 미니 일괄 처리에 대한 설정이 실패합니다.
- `logging_level`: 로그 세부 정보 수준입니다. 허용되는 값은 `warning`, `info` 및 `debug`입니다. 

다음 셀을 실행하면 배포가 구성되고 생성됩니다.

In [ ]:
<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 중요! 계속하기 전에 배포가 완료될 때까지 기다립니다. 스튜디오에 녹색 알림이 표시됩니다. </p>

엔드포인트에 여러 모델을 배포할 수 있습니다. 일괄 처리 엔드포인트를 호출할 때 기본적으로 사용해야 하는 모델을 지정하도록 기본 배포를 설정할 수 있습니다.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 중요! 계속하기 전에 기본 배포가 설정될 때까지 기다립니다. 스튜디오에 녹색 알림이 표시됩니다. </p>

## 일괄 처리 예측을 위한 데이터 준비

폴더에서 레이블이 `data` 지정되지 않은 데이터가 있는 CSV 파일을 찾을 수 있습니다. 폴더의 파일을 `data` 가리키는 데이터 자산을 만듭니다. 이 자산은 일괄 처리 작업에 대한 입력으로 사용합니다.

In [ ]:
## 작업 제출

이제 모델을 일괄 처리 엔드포인트에 배포하고 레이블이 지정되지 않은 데이터 자산을 갖게 되었으므로 레이블이 지정되지 않은 데이터에 대한 예측을 생성하기 위해 엔드포인트를 호출할 준비가 되었습니다.

먼저 등록된 데이터 자산을 참조하여 입력을 정의합니다. 그런 다음, 파이프라인 작업을 제출하는 엔드포인트를 호출합니다. 작업 URL을 사용하여 Studio에서 모니터링할 수 있습니다. 작업에는 예측을 가져오기 위해 (생성된) 채점 스크립트의 실행을 나타내는 자식 작업이 포함됩니다.

## 결과 가져오기

일괄 처리 엔드포인트를 호출하는 파이프라인 작업이 완료되면 결과를 볼 수 있습니다. 모든 예측은 기본 데이터 저장소에 `predictions.csv` 저장된 파일에 수집됩니다. 다음 셀을 실행하여 파일을 다운로드하고 데이터를 시각화할 수 있습니다. 

In [ ]:
from azure.ai.ml.entities import BatchEndpoint

# create a batch endpoint
endpoint = BatchEndpoint(
    name=endpoint_name,
    description="A batch endpoint for classifying diabetes in patients",
)

ml_client.batch_endpoints.begin_create_or_update(endpoint)

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the endpoint is created before continuing! A green notification should appear in the studio. </p>

## Create the deployment

A deployment is a set of resources required for hosting the model that does the actual inferencing. We will create a deployment for our endpoint using the `BatchDeployment` class. 

Since you're deploying an MLflow model, you don't need a scoring script or define the environment. Azure Machine Learning will automatically create those assets for you. The `MLmodel` file in the `model` folder is used to understand what the expected inputs and outputs are of the model.

You'll deploy a model with the following parameters:

- `name`: Name of the deployment.
- `description`: Optional description to further clarify what the deployment represents.
- `endpoint_name`: Name of the previously created endpoint the model should be deployed to.
- `model`: Name of the registered model.
- `compute`: Compute to be used when invoking the deployed model to generate predictions.
- `instance_count`: Count of compute nodes to use for generating predictions.
- `max_concurrency_per_instance`: Maximum number of parallel scoring script runs per compute node.
- `mini_batch_size`: Number of files passed per scoring script run.
- `output_action`: Each new prediction will be appended as a new row to the output file.
- `output_file_name`: File to which predictions will be appended.
- `retry_settings`: Settings for a mini-batch fails.
- `logging_level`: The log verbosity level. Allowed values are `warning`, `info`, and `debug`. 

Running the following cell will configure and create the deployment.

In [ ]:
from azure.ai.ml.entities import BatchDeployment, BatchRetrySettings
from azure.ai.ml.constants import BatchDeploymentOutputAction

deployment = BatchDeployment(
    name="classifier-diabetes-mlflow",
    description="A diabetes classifier",
    endpoint_name=endpoint.name,
    model=model,
    compute="aml-cluster",
    instance_count=2,
    max_concurrency_per_instance=2,
    mini_batch_size=2,
    output_action=BatchDeploymentOutputAction.APPEND_ROW,
    output_file_name="predictions.csv",
    retry_settings=BatchRetrySettings(max_retries=3, timeout=300),
    logging_level="info",
)
ml_client.batch_deployments.begin_create_or_update(deployment)

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the deployment is completed before continuing! A green notification should appear in the studio. </p>

You can deploy multiple models to an endpoint. You can set the default deployment to specify which model should be used by default when calling a batch endpoint.

In [ ]:
endpoint.defaults = {}

endpoint.defaults["deployment_name"] = deployment.name

ml_client.batch_endpoints.begin_create_or_update(endpoint)

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the default deployment is set before continuing! A green notification should appear in the studio. </p>

## Prepare the data for batch predictions

In the `data` folder you'll find CSV files with unlabeled data. You'll create a data asset that points to the files in the `data` folder, which you'll use as input for the batch job.

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

data_path = "./data"
dataset_name = "patient-data-unlabeled"

patient_dataset_unlabeled = Data(
    path=data_path,
    type=AssetTypes.URI_FOLDER,
    description="An unlabeled dataset for diabetes classification",
    name=dataset_name,
)
ml_client.data.create_or_update(patient_dataset_unlabeled)

In [ ]:
patient_dataset_unlabeled = ml_client.data.get(
    name="patient-data-unlabeled", label="latest"
)

## Submit the job

Now that you have deployed a model to a batch endpoint, and have an unlabeled data asset, you're ready to invoke the endpoint to generate predictions on the unlabeled data.

First, you'll define the input by referring to the registered data asset. Then, you'll invoke the endpoint, which will submit a pipeline job. You can use the job URL to monitor it in the Studio. The job will contain a child job that represents the running of the (generated) scoring script to get the predictions.

In [ ]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes

input = Input(type=AssetTypes.URI_FOLDER, path=patient_dataset_unlabeled.id)

In [ ]:
job = ml_client.batch_endpoints.invoke(
    endpoint_name=endpoint.name, 
    input=input)

ml_client.jobs.get(job.name)

## Get the results

When the pipeline job that invokes the batch endpoint is completed, you can view the results. All predictions are collected in the `predictions.csv` file that is stored in the default datastore. You can download the file and visualize the data by running the following cells. 

In [ ]:
ml_client.jobs.download(name=job.name, download_path=".", output_name="score")

In [ ]:
with open("predictions.csv", "r") as f:
    data = f.read()

In [ ]:
from ast import literal_eval
import pandas as pd

score = pd.DataFrame(
    literal_eval(data.replace("\n", ",")), columns=["file", "prediction"]
)
score